In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    OpenAI.api_key = openai_api_key
client = OpenAI()


import pandas as pd
import numpy as np
import time

In [4]:
# CSVファイルを読み込む
idol_list = pd.read_csv('../../data/master/idolname_20240829.csv', encoding='utf-8')

# 'idol_group_name'カラムのテキストをリスト化し、欠損値を空文字に置換
texts = idol_list['idol_group_name'].fillna('').tolist()

# OpenAI APIを使ってテキストをエンベディング
def get_embedding(text, model='text-embedding-3-small'):
    try:
        response = client.embeddings.create(
            model=model,
            input=[text]  # テキストセグメントをリストに変換
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error: {e}")
        return None

# レート制限を考慮して、各テキストに対してエンベディングを取得
embeddings = []
for text in texts:
    embedding = get_embedding(text)
    if embedding is not None:
        embeddings.append(embedding)
    else:
        embeddings.append([0]*1536)  # エンベディングサイズに合わせてゼロベクトルを挿入
    time.sleep(0.1)  # レート制限を避けるために待機

In [14]:
idol_list["embedding"]=embeddings
idol_list.to_csv('../../data/master/idolname_embedding.csv', encoding='utf-8', index=False)

In [21]:
def normalize_l2(X,dim):
    X = np.array(X)[:,:dim]
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    # ノルムがゼロの要素を特定
    zero_norms = (norms == 0)
    # ゼロ除算を防ぐためにノルムがゼロの要素を1に置換
    norms[zero_norms] = 1
    # 各ベクトルを対応するノルムで割る
    X_normalized = X / norms
    # 元のノルムがゼロだったベクトルを元のままに戻す
    X_normalized[zero_norms.flatten()] = X[zero_norms.flatten()]
    return X_normalized

cut_dim = embeddings
norm_dim = normalize_l2(cut_dim,100)

In [22]:
norm_dim.shape

(6010, 100)

In [25]:
idolname_embedding_data = pd.DataFrame(norm_dim,index=texts)#100次元
idolname_embedding_data.index.name = "idol_group_name"
idolname_embedding_data.to_csv('../../data/master/idolname_embedding_data.csv', encoding='utf-8')